<a href="https://colab.research.google.com/github/Kavya2099/Customer-Churn-Prediction/blob/main/Churn_prediction_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

get data

In [37]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv'

!wget $data -O data-week-3.csv 

df= pd.read_csv('data-week-3.csv')

df.head()

--2022-11-23 20:34:00--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-03-churn-prediction/WA_Fn-UseC_-Telco-Customer-Churn.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 977501 (955K) [text/plain]
Saving to: ‘data-week-3.csv’

data-week-3.csv     100%[===================>] 954.59K  --.-KB/s    in 0.06s   

2022-11-23 20:34:00 (16.0 MB/s) - ‘data-week-3.csv’ saved [977501/977501]



,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [38]:
df.shape

df.columns= df.columns.str.replace(" ","_").str.lower()
df.columns

strings= list(df.dtypes[df.dtypes=='object'].index)

for col in strings:
  df[col]= df[col].str.replace(" ","_").str.lower()

df.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,7590-vhveg,female,0,yes,no,1,no,no_phone_service,dsl,no,...,no,no,no,no,month-to-month,yes,electronic_check,29.85,29.85,no
1,5575-gnvde,male,0,no,no,34,yes,no,dsl,yes,...,yes,no,no,no,one_year,no,mailed_check,56.95,1889.5,no
2,3668-qpybk,male,0,no,no,2,yes,no,dsl,yes,...,no,no,no,no,month-to-month,yes,mailed_check,53.85,108.15,yes
3,7795-cfocw,male,0,no,no,45,no,no_phone_service,dsl,yes,...,yes,yes,no,no,one_year,no,bank_transfer_(automatic),42.30,1840.75,no
4,9237-hqitu,female,0,no,no,2,yes,no,fiber_optic,no,...,no,no,no,no,month-to-month,yes,electronic_check,70.70,151.65,yes


In [39]:
df.dtypes


customerid           object
gender               object
seniorcitizen         int64
partner              object
dependents           object
tenure                int64
phoneservice         object
multiplelines        object
internetservice      object
onlinesecurity       object
onlinebackup         object
deviceprotection     object
techsupport          object
streamingtv          object
streamingmovies      object
contract             object
paperlessbilling     object
paymentmethod        object
monthlycharges      float64
totalcharges         object
churn                object
dtype: object

Changing the data type of monthly charges and churn columns to int

In [40]:
df.monthlycharges = df.monthlycharges.astype('int')
df.monthlycharges
df.churn = (df.churn=='yes').astype('int')
df.monthlycharges.dtype

dtype('int64')

Setting up the validation framework

spliting train, val and test data sets

In [41]:
X= df.drop(['churn'],axis=1)
y= df.churn

from sklearn.model_selection import train_test_split

def split(X,y):


  full_X_train, X_val, full_y_train, y_val = train_test_split(X,y,test_size=0.2, random_state=0)
  X_train, X_test, y_train , y_test = train_test_split(full_X_train,full_y_train,test_size=0.25, random_state=0)
  return (X_train, X_val, y_train , y_val)




In [42]:
#len(X_train),len(X_val),len(X_test)

Exploratory data analysis

Checking for missing values

In [43]:
df.isnull().sum()

customerid          0
gender              0
seniorcitizen       0
partner             0
dependents          0
tenure              0
phoneservice        0
multiplelines       0
internetservice     0
onlinesecurity      0
onlinebackup        0
deviceprotection    0
techsupport         0
streamingtv         0
streamingmovies     0
contract            0
paperlessbilling    0
paymentmethod       0
monthlycharges      0
totalcharges        0
churn               0
dtype: int64

There is no null values.
1st method. Doing random forest by taking all features

In [44]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score

#defining random forest model
def randommodel(X_train, X_val, y_train , y_val):
  model1= RandomForestClassifier(random_state=0)
  model1.fit(X_train, y_train)
  pred = model1.predict(X_val)

  
  print('ACCURACY: ',accuracy_score(y_val,pred)*100)
  

# calling model with all features
X= df.drop(['churn'],axis=1)
y= df.churn
X_train, X_val, y_train , y_val = split(X,y)
#model(X_train, X_val, y_train , y_val)




As we can see we are getting error:
1.   ValueError: could not convert string to float: '7544-zvikx'

So we have to either drop this feature (customer id)or factorize/hot encode  the feature



In [45]:
df.head()

,customerid,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,...,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,7590-vhveg,female,0,yes,no,1,no,no_phone_service,dsl,no,...,no,no,no,no,month-to-month,yes,electronic_check,29,29.85,0
1,5575-gnvde,male,0,no,no,34,yes,no,dsl,yes,...,yes,no,no,no,one_year,no,mailed_check,56,1889.5,0
2,3668-qpybk,male,0,no,no,2,yes,no,dsl,yes,...,no,no,no,no,month-to-month,yes,mailed_check,53,108.15,1
3,7795-cfocw,male,0,no,no,45,no,no_phone_service,dsl,yes,...,yes,yes,no,no,one_year,no,bank_transfer_(automatic),42,1840.75,0
4,9237-hqitu,female,0,no,no,2,yes,no,fiber_optic,no,...,no,no,no,no,month-to-month,yes,electronic_check,70,151.65,1


As we can see there are lot of catergorical variables and numerical variables in df. 

1.   So we'll split them separately and for categorical features we will only use categorical features which has less than 10 unique features, since more than that may not be appear in validation/test test and it can create error




In [46]:
# splitting numerical and categorical features


num_cols = [col for col in X_train.columns if X_train[col].dtype in ['int64','float64']]
print(num_cols)

cat_cols= [col for col in X_train.columns if (X_train[col].dtype == 'object' and X_train[col].nunique() < 10)]
print(cat_cols)

#We'll only use these columns to train the model and validate the model

cols = num_cols + cat_cols
X_train_1 = X_train[cols]
X_val_1 = X_val[cols]


['seniorcitizen', 'tenure', 'monthlycharges']
['gender', 'partner', 'dependents', 'phoneservice', 'multiplelines', 'internetservice', 'onlinesecurity', 'onlinebackup', 'deviceprotection', 'techsupport', 'streamingtv', 'streamingmovies', 'contract', 'paperlessbilling', 'paymentmethod']


Still, our problem hasn't solved, so now we'll train the model only using numerical feature and check the error output

In [47]:
X = df[num_cols]
y = df['churn']

#calling model1 with using numerical features

X_train, X_val, y_train , y_val = split(X,y)
randommodel(X_train, X_val, y_train , y_val)

ACCURACY:  74.59190915542938


We got accuracy to be 74.59190915542938 for random forest classifier model.
Now we'll try the same with logistic regression model

In [48]:
from sklearn.linear_model import LogisticRegression  

#defining logistic model
def logisticmodel(X_train, X_val, y_train , y_val):
  model1= LogisticRegression(max_iter=100)
  model1.fit(X_train, y_train)
  pred = model1.predict(X_val)

  
  print('ACCURACY: ',accuracy_score(y_val,pred)*100)

In [49]:
X = df[num_cols]
y = df['churn']

#calling model1 with using numerical features

X_train, X_val, y_train , y_val = split(X,y)
logisticmodel(X_train, X_val, y_train , y_val)

ACCURACY:  77.9985805535841


In [50]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier()
#one hot encoding
X = df[num_cols]
y = df['churn']
X_train, X_val, y_train , y_val = split(X,y)

xgb_model.fit(X_train, y_train)
prediction_xgb = xgb_model.predict(X_val)
print('ACCURACY: ',accuracy_score(y_val, prediction_xgb)*100)

ACCURACY:  78.49538679914833


So first level analysis

with only using numerical features

1. Random forest classifier = 74.59190915542938
2. Logistic regression = 77.9985805535841
3. XGB classifier = **78.49538679914833**

Other ways to improve the model 

We'll be using feature importance analysis topics to impove the accuracy

Mutual Information

In [51]:
#changing object dtype to int type using factorize 
X = df.drop(['churn'],axis=1)
mi= X
for col in mi.select_dtypes('object'):
  mi[col],_ = mi[col].factorize()

discrete_feature= mi.dtypes == int

from sklearn.feature_selection import mutual_info_classif

mi_scores = mutual_info_classif(mi,y)
mi_scores= pd.Series(mi_scores, name='mi_scores',index=mi.columns)
mi_scores= mi_scores.sort_values(ascending=False)
print(mi_scores)

contract            0.098234
tenure              0.078001
onlinesecurity      0.062190
techsupport         0.060203
onlinebackup        0.057104
internetservice     0.051725
monthlycharges      0.045580
paymentmethod       0.043483
deviceprotection    0.043367
streamingtv         0.040651
streamingmovies     0.032840
paperlessbilling    0.020188
partner             0.013374
multiplelines       0.006692
seniorcitizen       0.006645
dependents          0.004805
phoneservice        0.004551
gender              0.002124
customerid          0.000000
totalcharges        0.000000
Name: mi_scores, dtype: float64


Checking correlation with numerical variables

In [52]:
# numerical variables

df[num_cols].corrwith(df.churn)

seniorcitizen     0.150889
tenure           -0.352229
monthlycharges    0.193422
dtype: float64

This shows tenure has negative coreleation with churn rate and seniorcitizen and monthly charges has positive corelation with churn rate



1.   High churn low tenure
2.   High churn high monthly charges/senior citizen



Also, we have verified that features that has high MI is included in the cols[num+cat]

Next, we'll try XGB classifier with the num+cat

### **XGB classifier with the num+cat**

In [53]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier()
#one hot encoding
X= df[cols]
X = pd.get_dummies(X, columns=cols)
X_train, X_val, y_train , y_val = split(X,y)

xgb_model.fit(X_train, y_train)
prediction_xgb = xgb_model.predict(X_val)
print('ACCURACY: ',accuracy_score(y_val, prediction_xgb)*100)

ACCURACY:  79.4889992902768


XGB classifier gives 79.4889992902768

### **RF classifier with the num+cat**

In [54]:
X= df[cols]
X = pd.get_dummies(X, columns=cols)
print(X.shape)
X_train, X_val, y_train , y_val = split(X,y)
print('random forest classifier')
randommodel(X_train, X_val, y_train , y_val)


(7043, 215)
random forest classifier
ACCURACY:  76.72107877927608


### **Logistic Regression with the num+cat**

In [55]:
X= df[cols]
X = pd.get_dummies(X, columns=cols)
X_train, X_val, y_train , y_val = split(X,y)
print('logistic regressor')
logisticmodel(X_train, X_val, y_train , y_val)

logistic regressor
ACCURACY:  78.99219304471256


We'll just check what will be the case if we use all the features in XGB classifier instead of just all columns

### **XGB with all columns**

In [56]:
from xgboost import XGBClassifier
xgb_model = XGBClassifier()
#one hot encoding
X=df.drop(['churn'],axis=1)
X = pd.get_dummies(X)
X_train, X_val, y_train , y_val = split(X,y)

xgb_model.fit(X_train, y_train)
prediction_xgb = xgb_model.predict(X_val)
print('ACCURACY: ',accuracy_score(y_val, prediction_xgb)*100)

ACCURACY:  78.85024840312278


We'll just the same for RF and logistic regression

### **RF with all columns**

In [57]:
X=df.drop(['churn'],axis=1)
X = pd.get_dummies(X)
X_train, X_val, y_train , y_val = split(X,y)
print('random forest classifier')
randommodel(X_train, X_val, y_train , y_val)


random forest classifier
ACCURACY:  78.49538679914833


### **Logistic Regression with all columns**

In [58]:
X=df.drop(['churn'],axis=1)
X = pd.get_dummies(X)
print(X.shape)
X_train, X_val, y_train , y_val = split(X,y)
print('logistic regressor')
logisticmodel(X_train, X_val, y_train , y_val)

(7043, 13618)
logistic regressor
ACCURACY:  79.13413768630234


***With only using numerical features***

 Random forest classifier = 74.59190915542938
 
 Logistic regression = 77.9985805535841

 XGB classifier = 78.49538679914833



---


---


***With only using specific features***

Random forest classifier = 76.72107877927608

Logistic regression = 78.99219304471256

**XGB classifier = 79.4889992902768**


---





---

***With all features***

Random forest classifier = 78.49538679914833

Logistic regression = 79.13413768630234

XGB classifier = 78.85024840312278

lets test the model

we'll use pipeline with logistic regression as it gave the max accuracy

In [68]:

X = df[cols]
X = pd.get_dummies(X, columns=cols)
#print(X.columns)
y = df['churn']

#calling model1 with using numerical features

full_X_train, X_val, full_y_train, y_val = train_test_split(X,y,test_size=0.2, random_state=0)
X_train, X_test, y_train , y_test = train_test_split(full_X_train,full_y_train,test_size=0.25, random_state=0)
  

print('logistic regressor')
model1= LogisticRegression(max_iter=100)
model1.fit(full_X_train, full_y_train)
pred = model1.predict(X_test)
#assigning pred to p variable to get the mean
p=pred
  
print('ACCURACY: ',accuracy_score(y_test,pred)*100)


logistic regressor
ACCURACY:  81.83108587650815


Accuracy with test set 81.83108587650815



Now lets check with single data

In [61]:
# 10th data

customer = X_test.iloc[10]
customer

seniorcitizen_0                            1
seniorcitizen_1                            0
tenure_0                                   0
tenure_1                                   0
tenure_2                                   0
                                          ..
paperlessbilling_yes                       1
paymentmethod_bank_transfer_(automatic)    0
paymentmethod_credit_card_(automatic)      0
paymentmethod_electronic_check             1
paymentmethod_mailed_check                 0
Name: 1253, Length: 215, dtype: uint8

In [62]:
customer.shape
#(18,)
# the reason we are changing it to dataframe is that single data record is considered as 18 rows and is in 1d. We cannot use that to predict the value. 
#So we are changing it to 2d using dataframe
df_small = pd.DataFrame([customer])
df_small.shape

(1, 215)

In [63]:
preds = model1.predict(df_small)

preds= int(preds)
print("predicted value: ",preds)
print("actual value: ",y_test.iloc[10])





predicted value:  1
actual value:  1


We'll check for the last data

In [64]:
customer = X_test.iloc[-1]

df_small = pd.DataFrame([customer])
df_small.shape

preds = model1.predict(df_small)
preds= int(preds)
print("predicted value: ",preds)
print("actual value: ",y_test.iloc[-1])

predicted value:  0
actual value:  0


In [65]:
y_test.mean()

0.2590489709013485

In [66]:
1- y_test.mean()

0.7409510290986515

in test data it is said that, 25.9 % people are likely to churn and 74% are not churning

Lets check for model predicted value

In [69]:
p.mean()

0.20227111426543648

In [70]:
1- p.mean()

0.7977288857345635

In our model, 20.2 % people are likely to churn and 79.7 % are not churning